## Re-using the A=PLU factorization

In some situations we can re-use the A=PLU factorization in multiple problems. An example is solving a boundary value problem for several different boundary conditions. We describe a example of this: the discrete Dirichlet problem.

## The discrete Laplacian for a graph.

Given a graph with $n$ labelled vertices we define the Adjacency matrix $A$  to be the $n\times n$ matrix with matrix entries

$$
A_{i,j} = 1 \quad\hbox{ if }\quad i \sim j,\quad  \hbox{otherwise } 0
$$
Here $n\sim n$ means $i$ and $j$ are adjacent, i.e., nearest neighbours. 

The discrete Laplacian is the $n\times n$ matrix with matrix entries
$$
L_{i,j} =  A_{i,j} - \delta_{i,j} c(i)
$$
where $c(i)$ is the number of edges meeting $i$.

We identify a function $f$ on the vertices with the vector $\begin{bmatrix}f_1\\ f_2\\ \vdots\\ f_n\\\end{bmatrix}$ in $\mathbb R^n$ that lists the values at each vertex. 

Then a more natural (eqivalent) definition of $L$ is in terms its action as a linear transformation:
$$
(Lf)_i = \sum_{j\sim i}(f_j-f_i)
$$
If we think of $(f_j-f_i)$ as the change in $f$ when you move along the edge joining $i$ to $j$, then $(Lf)_i$ is the sum over all edges joining $i$ of these changes. You should convince yourself that the two definitions agree. The first one is easier to compute.

Here is an example: For the graph

![title](img/dd.png)

$ A = \begin{bmatrix} 0 & 1 & 0 & 0 & 1\\   1& 0 &1&0  &1\\0   &1 &0  & 1&1\\   0&0&1&0&1\\  1 &1&1&1&0\\\end{bmatrix}$


$L= \begin{bmatrix} -2 & 1 & 0 & 0 & 1\\   1& -3 &1&0  &1\\0   &1 &-3  & 1&1\\   0&0&1&-2&1\\  1 &1&1&1&-4\\\end{bmatrix}$

## Boundary conditions and the Dirichlet problem


Let's designate some sunset of vertices to be the boundary. Suppose that there are $m$ of them. By relabelling if needed, we may assume that the labels $0, 1, 2, \ldots m-1$ correspond to the boundary and the labels $m, \ldots, n-1$ correspond to the interior.

![title](img/db.png)

In the example graph the blue vertices  $0, 1$  are the boundary. So $m=2$.

Let
$$
P= [I_{m} | 0]
$$
where $I_{m}$ is the $m\times m$ identity matrix and $0$ denotes an $m\times (n-m)$ block of zeros. 
Similarly let
$$
Q=[0|I_{n-m}]
$$
If $f\in \mathbb R^n$ denotes a function on the vertices, then $Pf$ is the restriction of $f$ to the boundary and $Qf$ is the restriction of $f$ to interior points. Notice that
$$
P^T P + Q^T Q = I
$$

## The Dirichlet problem 

Given a function $b$ defined on the boundary, find a function $f$ defined on all vertices such that

1. $f=b$ for all boundary points

2. $Lf = 0$ for all interior points. 

In our notation this translates to 

Given $b\in \mathbb R^m$, find $f\in \mathbb R^n$ such that 

1. $Pf = b$
2. $QLf = 0$$

We solve this by inserting $P^T P + Q^T Q = $i$ into the second equation. 

This gives 

$$QLF = QL(P^T P  +  Q^TQ)f = QLP^T Pf + QLQ^T Qf =0.$$ 

If the first equation holds as well then $QLQ^T QF = -QLP^Tb$ so 
$$
QLQ^T Qf = -QLP^Tb
$$
This equation is of the form $ Ax = c$ where $A=QLQ^T$, $x=Qf$ and $c=-QLP^Tb$. If we can solve it then we know $x=Qf$ and $b=Pf$ so we can construct 

$$f=( P^T P + Q^T Q )f  = P^Tb + Q^T x.$$


## Main point: Re-using the A=PLU factorization when we change boundary condition

Now suppose we want to solve the Dirichlet problem for a bunch of different boundary values $b_1, b_2, ...$ on the same graph with boundary. Then we need to solve  $Ax= c_1, Ax=c_2, \ldots $ 

We can solve each of these equations in two steps: 

(1) Find the A=PLU decomposition of $A$

(2) Solve the factored equation $PLU x = c_i$ where $c_i=-QLP^Tb_i$.

Step (1) is where most of the work is. But this step is the *same* for each of our equations. So we need only do this step once and then use the result to do step (2) for each equation.

Lets do these steps in python for the example graph and boundary value $b=\begin{bmatrix}1\\0\\ \end{bmatrix}$

In [24]:
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt

L = np.array([[-2, 1, 0, 0, 1],[1, -3 ,1 ,0 ,1],[0,1,-3,1,1],[0,0,1 ,-2,1],[1,1,1,1,-4]])
P = np.array([[1,0,0,0,0],[0,1,0,0,0]])
Q = np.array([[0,0,1,0,0],[0,0,0,1,0],[0,0,0,0,1]])
A = Q@L@np.transpose(Q)
b = np.array([[1],[0]])
c = -Q@L@np.transpose(P)@b
print("L = \n", L)
print("P = \n", P)
print("Q = \n", Q)
print("A = \n", A)
print("b = \n", b)
print("c = \n", c)

L = 
 [[-2  1  0  0  1]
 [ 1 -3  1  0  1]
 [ 0  1 -3  1  1]
 [ 0  0  1 -2  1]
 [ 1  1  1  1 -4]]
P = 
 [[1 0 0 0 0]
 [0 1 0 0 0]]
Q = 
 [[0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]]
A = 
 [[-3  1  1]
 [ 1 -2  1]
 [ 1  1 -4]]
b = 
 [[1]
 [0]]
c = 
 [[ 0]
 [ 0]
 [-1]]


We could do this:

In [30]:
x = la.solve(A,c)
f = np.transpose(P)@b+np.transpose(Q)@x
print("f=",f)

f= [[1.        ]
 [0.        ]
 [0.23076923]
 [0.30769231]
 [0.38461538]]


If we are only doing this problem for one value of $b$ this is fine. But if we want to solve the equation for different $b$'s We should get the solution via the LU factorization

In [29]:
lu, piv = la.lu_factor(A)
x = la.lu_solve((lu,piv),c)
f = np.transpose(P)@b+np.transpose(Q)@x
print("f=",f)

f= [[1.        ]
 [0.        ]
 [0.23076923]
 [0.30769231]
 [0.38461538]]


The advantage is that if we now want to solve the Dirichlet problem with a new boundary condition, say,
$b_2 = b=\begin{bmatrix}0\\1\\ \end{bmatrix}$. Then we take
$c2 = -Q L P^Tb_2$ and reuse lu and piv

In [35]:
b2 = np.array([[0],[1]])
c2 = -Q@L@np.transpose(P)@b2
x2 = la.lu_solve((lu,piv),c2)
f2=np.transpose(P)@b2+np.transpose(Q)@x2
print("f2=",f2)

f2= [[0.        ]
 [1.        ]
 [0.76923077]
 [0.69230769]
 [0.61538462]]


**Homework problem 1: The discrete Poisson kernel is the $5\times 2$ matrix $PK$ such that for any boundary condtion $b_2$, $b_3$, the product $PK  \begin{bmatrix}b_2\\b_3\\\end{bmatrix}$ is the solution of the Dirichlet problem with this boundary condition. 
Compute $PK$ for the example above**
Hint: recall that if a linear transformation is given by a matrix we can find the maxtrix by applying the linear transformation to standard basis vectors. For example, for a nx2 matrix R we find the columns of R by computing R [1,0]^T (first row) and R [0,1]^T (second row)

To find $PK$ simply stack the results for $f$ and $f_2$ obtained above, which are the first/second rows of the matrix.
Here's all the code in a more organized fashion.

In [3]:
import numpy as np
import scipy.linalg as la

L = np.array([
    [-2, 1, 0, 0, 1],
    [1, -3, 1, 0, 1],
    [0, 1, -3, 1, 1],
    [0, 0, 1, -2, 1],
    [1, 1, 1, 1, -4]
])
P = np.array([
    [1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0]
])
Q = np.array([
    [0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0],
    [0, 0, 0, 0, 1]
])

A = Q @ L @ Q.T
lu, piv = la.lu_factor(A)

bs, fs = [np.array([1, 0]).T, np.array([0, 1]).T], []
for b in bs:
    c = -Q @ L @ P.T @ b
    x = la.lu_solve((lu, piv), c)
    fs.append(P.T @ b + Q.T @ x)

PK = np.vstack(fs)
print(f'PK = {PK}')

PK = [[1.         0.         0.23076923 0.30769231 0.38461538]
 [0.         1.         0.76923077 0.69230769 0.61538462]]


**Homework problem 2:
For the graph with boundary shown here**

![title](img/d3.png)

**Compute the solution of the Dirichlet problem  with boundary values $b_0=1$, $b_1=b_2=\cdots = b_5=0$ using the LU decomposition. Then reuse the decomposition to solve the Dirichlet problem  with boundary values $b_0=b_1=b_2=\cdots = b_5=1$
and then with values $b_0= 0, b_1=1 , b_2=b_3\cdots = b_5=0$. (Actually there is an even easier way to get this last one using the symmetry. Use this to check your computed answer.)**

We write the Laplacian for this graph by hand and then use the PLU factorization method to solve the 3 BVPs.
The last BVP is identical to first BVP under the following vertex mappings, which can be represented as a permutation matrix.
$
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}\rightarrow 1, 2, 3, 4, 5, 0, 7, 10, 6, 9, 12, 8, 11
$

The last solution $f_3$ agrees reasonably well with the first solution $f_1$ (after permuted).

The second solution $f_2$ also makes sense. Since the solution to a Dirichlet problem with constant BVs $1$ is the constant function $1$.

In [8]:
import numpy as np
import scipy.linalg as la

L = np.array([
    [-2, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
    [0, -2, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0],
    [0, 0, -2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1],
    [0, 0, 0, -2, 0, 0, 0, 0, 0, 0, 0, 1, 1],
    [0, 0, 0, 0, -2, 0, 0, 0, 1, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, -2, 0, 0, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 0, 1, -5, 1, 1, 1, 0, 0, 0],
    [1, 1, 0, 0, 0, 0, 1, -5, 0, 1, 1, 0, 0],
    [0, 0, 0, 0, 1, 1, 1, 0, -5, 1, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 1, 1, 1, -6, 1, 1, 1],
    [0, 1, 1, 0, 0, 0, 0, 1, 0, 1, -5, 0, 1],
    [0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, -5, 1],
    [0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, -5]
])
P = np.hstack([np.eye(6), np.zeros((6, 7))])
Q = np.hstack([np.zeros((7, 6)), np.eye(7)])

A = Q @ L @ Q.T
lu, piv = la.lu_factor(A)

b1 = np.array([1, 0, 0, 0, 0, 0]).T
b2 = np.ones(6).T
b3 = np.array([0, 1, 0, 0, 0, 0]).T
bs, fs = [b1, b2, b3], []
for b in bs:
    c = -Q @ L @ P.T @ b
    x = la.lu_solve((lu, piv), c)
    fs.append(P.T @ b + Q.T @ x)

for i in range(3):
    print(f'f{i + 1} = {fs[i]}')



f1 = [1.         0.         0.         0.         0.         0.
 0.31039165 0.31182104 0.09348199 0.14665523 0.10205832 0.01036306
 0.05181532]
f2 = [1.         1.         1.         1.         1.         1.
 0.9348199  0.94511149 0.87307033 0.85591767 0.9348199  0.57461407
 0.87307033]
f3 = [0.         1.         0.         0.         0.         0.
 0.10341624 0.31296455 0.05445969 0.14965695 0.31174957 0.01922527
 0.09612636]
